## Disclaimer:

### O notebook em questão possui como objetivo a vizualisação e exploração de dados dos estudantes da Universidade Federal de Lavras (UFLA). Os dados em questão estão disponíveis para consulta pública [aqui](https://dados.gov.br/dados/conjuntos-dados/estudantes-da-graduacao). A ferramenta utilizada para manipulação dos dados foi o `PySpark`.

In [2]:
# Criação da sessão Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("data_exploration").getOrCreate()
import warnings
warnings.filterwarnings('ignore')

24/02/23 10:07:28 WARN Utils: Your hostname, murilo resolves to a loopback address: 127.0.1.1; using 172.25.92.128 instead (on interface wlp0s20f3)
24/02/23 10:07:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/23 10:07:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### SETUP

In [4]:
# IMPORTS
import random
import pyspark.pandas as ps
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import rand, when

### DATA VIEW

In [5]:
# Carregando os dados como um objeto PySpark
data = spark.read.csv('dados_abertos___estudantes_da_graduacao_2024-02-22T12_08_52.899842-03_00.csv', header=True, inferSchema=True)
type(data)

pyspark.sql.dataframe.DataFrame

In [6]:
# Tabela com os dados
data.show()

+------------------+--------------------+--------------------------+---------------------+------------------+------------------------+--------------+-------------------+-------------------+-----------+-----------+----------------------+------------------+----------------+-----------------+--------------------+--------------+--------------+--------------+--------------------+
|registro_academico|               curso|semestre_letivo_referencia|ano_letivo_referencia|situacao_estudante|semestre_letivo_ingresso|forma_ingresso|forma_ingresso_tipo|codigo_oferta_curso|codigo_inep|curso_turno|nome_matriz_curricular|  titulo_academico|curso_modalidade|    situacao_nome|  situacao_descricao|situacao_censo|        campus|sistema_origem| data_atualizacao_dw|
+------------------+--------------------+--------------------------+---------------------+------------------+------------------------+--------------+-------------------+-------------------+-----------+-----------+----------------------+--------

In [7]:
# Para plotar os gráficos no notebook
%matplotlib inline

In [8]:
# Para melhor manipulação trasnforma o objeto em SparkPandas
view_1 = ps.DataFrame(data.groupBy('semestre_letivo_referencia', 'situacao_descricao')
                      .count()
                      .orderBy('semestre_letivo_referencia'))

view_1_pivot = view_1.pivot(index='semestre_letivo_referencia', 
                            columns='situacao_descricao', 
                            values='count')
view_1_pivot.plot(kind='bar')

In [9]:
# Para melhor manipulação trasnforma o objeto em SparkPandas
view_2 = ps.DataFrame(data.groupBy('semestre_letivo_referencia', 'situacao_estudante')
                      .count()
                      .orderBy('semestre_letivo_referencia'))

view_2_pivot = view_2.pivot(index='semestre_letivo_referencia', 
                            columns='situacao_estudante', 
                            values='count')
view_2_pivot.plot(kind='bar')

In [10]:
view_2 = data.groupBy('situacao_estudante').count()
total = view_2.groupBy().agg(F.sum('count').alias('total')).collect()[0]['total']
view_2 = view_2.withColumn('percent', (F.col('count') / total) * 100)

view_2 = ps.DataFrame(view_2)
view_2 = view_2.set_index('situacao_estudante')
view_2.plot(kind='pie',y='percent')

In [11]:
# Para melhor manipulação trasnforma o objeto em SparkPandas
view_3 = ps.DataFrame(data.groupBy('semestre_letivo_referencia', 'curso')
                      .count()
                      .orderBy('semestre_letivo_referencia'))

view_3_pivot = view_3.pivot(index='semestre_letivo_referencia', 
                            columns='curso', 
                            values='count')
view_3_pivot.plot(kind='bar')

24/02/23 10:09:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
# Para melhor manipulação trasnforma o objeto em SparkPandas
view_4 = ps.DataFrame(data.groupBy('curso', 'curso_turno')
                      .count()
                      .orderBy('curso'))

view_4_pivot = view_4.pivot(index='curso', 
                            columns='curso_turno', 
                            values='count')
view_4_pivot.plot(kind='barh')

In [13]:
# Para melhor manipulação trasnforma o objeto em SparkPandas
view_5 = ps.DataFrame(data.groupBy('semestre_letivo_referencia', 'curso_turno')
                      .count()
                      .orderBy('semestre_letivo_referencia'))

view_5_pivot = view_5.pivot(index='semestre_letivo_referencia', 
                            columns='curso_turno', 
                            values='count')
view_5_pivot.plot(kind='bar')

### Tratamento dos dados:

In [14]:
valores_unicos_lista = data.select("situacao_descricao").distinct().collect()
valores_unicos_lista = [row["situacao_descricao"] for row in valores_unicos_lista]

In [15]:
situacao_descricao_to_pop = ['Discente trancou o curso por um período',
                            'Conclusão dos requisitos acadêmicos ou titulação',
                            'Aluno vinculado a UFLA, sem cursar disciplinas.',
                            'Discente transferido para outra instituição',
                            'Saiu do curso em função de desligamento por insuficiência ou outros',
                            'Discente solicitou explicitamente a saída do curso',
                            'Discente não compareceu ao curso por um período e, com isso, perdeu a vaga',
                            'Discente realizando Atividade Acadêmica Internacional, regulamentada pela Resolução CEPE 121/2014',
                            'Falecimento do discente',
                            'Programa de Mobilidade Estudantil',
                            'Discente desligado por não renovação de matrícula',
                            'Saiu do curso em função de desligamento por tempo (não concluiu o curso no tempo previsto)',
                            'Encerrou o período de trancamento geral no curso',
                            'Mudança do discente de um curso para uma habilitação ou de uma área básica para um curso/habilitação relacionados',
                            'Conclusão dos requisitos acadêmicos ou titulação com apostilamento',
                            ]

In [16]:
#Tratamento #1: Remoção de alunos com perfil inadequado
data = data.filter(~data["situacao_descricao"].isin(situacao_descricao_to_pop))

In [17]:
data.show()

+------------------+--------------------+--------------------------+---------------------+------------------+------------------------+--------------+-------------------+-------------------+-----------+-----------+----------------------+--------------------+----------------+-------------+--------------------+--------------+--------------+--------------+--------------------+
|registro_academico|               curso|semestre_letivo_referencia|ano_letivo_referencia|situacao_estudante|semestre_letivo_ingresso|forma_ingresso|forma_ingresso_tipo|codigo_oferta_curso|codigo_inep|curso_turno|nome_matriz_curricular|    titulo_academico|curso_modalidade|situacao_nome|  situacao_descricao|situacao_censo|        campus|sistema_origem| data_atualizacao_dw|
+------------------+--------------------+--------------------------+---------------------+------------------+------------------------+--------------+-------------------+-------------------+-----------+-----------+----------------------+------------

In [18]:
situacao_estudante_to_pop = ['Inativo']
data = data.filter(~data["situacao_estudante"].isin(situacao_estudante_to_pop))
data.show()

+------------------+--------------------+--------------------------+---------------------+------------------+------------------------+--------------+-------------------+-------------------+-----------+-----------+----------------------+--------------------+----------------+-------------+--------------------+--------------+--------------+--------------+--------------------+
|registro_academico|               curso|semestre_letivo_referencia|ano_letivo_referencia|situacao_estudante|semestre_letivo_ingresso|forma_ingresso|forma_ingresso_tipo|codigo_oferta_curso|codigo_inep|curso_turno|nome_matriz_curricular|    titulo_academico|curso_modalidade|situacao_nome|  situacao_descricao|situacao_censo|        campus|sistema_origem| data_atualizacao_dw|
+------------------+--------------------+--------------------------+---------------------+------------------+------------------------+--------------+-------------------+-------------------+-----------+-----------+----------------------+------------

In [19]:
# Schema dos dados
data.printSchema()

root
 |-- registro_academico: integer (nullable = true)
 |-- curso: string (nullable = true)
 |-- semestre_letivo_referencia: string (nullable = true)
 |-- ano_letivo_referencia: date (nullable = true)
 |-- situacao_estudante: string (nullable = true)
 |-- semestre_letivo_ingresso: string (nullable = true)
 |-- forma_ingresso: string (nullable = true)
 |-- forma_ingresso_tipo: string (nullable = true)
 |-- codigo_oferta_curso: string (nullable = true)
 |-- codigo_inep: integer (nullable = true)
 |-- curso_turno: string (nullable = true)
 |-- nome_matriz_curricular: integer (nullable = true)
 |-- titulo_academico: string (nullable = true)
 |-- curso_modalidade: string (nullable = true)
 |-- situacao_nome: string (nullable = true)
 |-- situacao_descricao: string (nullable = true)
 |-- situacao_censo: string (nullable = true)
 |-- campus: string (nullable = true)
 |-- sistema_origem: string (nullable = true)
 |-- data_atualizacao_dw: timestamp (nullable = true)



In [20]:
#Remoção de variáveis fora do contexto
columns_to_drop = ['situacao_estudante', 
                   'codigo_oferta_curso', 
                   'codigo_inep',
                   'nome_matriz_curricular', 
                   'titulo_academico', 
                   'situacao_nome', 
                   'situacao_descricao', 
                   'data_atualizacao_dw',
                   'sistema_origem']

data_cleaned = data.drop(*columns_to_drop)
data_cleaned.show()

+------------------+--------------------+--------------------------+---------------------+------------------------+--------------+-------------------+-----------+----------------+--------------+--------------+
|registro_academico|               curso|semestre_letivo_referencia|ano_letivo_referencia|semestre_letivo_ingresso|forma_ingresso|forma_ingresso_tipo|curso_turno|curso_modalidade|situacao_censo|        campus|
+------------------+--------------------+--------------------------+---------------------+------------------------+--------------+-------------------+-----------+----------------+--------------+--------------+
|         202325010|Bacharelado Inter...|                    2023/2|           2023-01-01|                  2023/2|   SiSU (Enem)|            Regular|   Integral|      Presencial|      Cursando|Campus Paraíso|
|         202325009|Bacharelado Inter...|                    2023/2|           2023-01-01|                  2023/2|   SiSU (Enem)|            Regular|   Integra

### ADIÇÃO DE DADOS SINTÉTICOS:

In [21]:
random.seed(0)

In [22]:
# Carregando os valores sintéticos
from course_mappings import course_skills, course_interests

In [23]:
def get_random_skills(course):
    skills = course_skills.get(course, [])
    return random.sample(skills, min(len(skills), 3))

def get_random_interests(course):
    interests = course_interests.get(course, [])
    return random.sample(interests, min(len(interests), 3))

get_random_skills_udf = udf(get_random_skills, ArrayType(StringType()))
get_random_interests_udf = udf(get_random_interests, ArrayType(StringType()))

In [24]:
# Aplicar as UDFs para adicionar os campos 'habilidades' e 'interesse'
data_cleaned = data_cleaned.withColumn('habilidades', get_random_skills_udf(data_cleaned['curso']))
data_cleaned = data_cleaned.withColumn('interesses', get_random_interests_udf(data_cleaned['curso']))

# Adição da flag de de estágio com 1 ou 0
data_cleaned = data_cleaned.withColumn('estagiando', when(rand() < 0.5, 1).otherwise(0))

In [25]:
data_cleaned.show()

+------------------+--------------------+--------------------------+---------------------+------------------------+--------------+-------------------+-----------+----------------+--------------+--------------+--------------------+--------------------+----------+
|registro_academico|               curso|semestre_letivo_referencia|ano_letivo_referencia|semestre_letivo_ingresso|forma_ingresso|forma_ingresso_tipo|curso_turno|curso_modalidade|situacao_censo|        campus|         habilidades|          interesses|estagiando|
+------------------+--------------------+--------------------------+---------------------+------------------------+--------------+-------------------+-----------+----------------+--------------+--------------+--------------------+--------------------+----------+
|         202325010|Bacharelado Inter...|                    2023/2|           2023-01-01|                  2023/2|   SiSU (Enem)|            Regular|   Integral|      Presencial|      Cursando|Campus Paraíso|[P

### Definição da Variável target e Clusterização dos dados

# TODO: FILTRAR OS DADOS DE ENTRADA PARA OS FORMATOS E CAMPOS DE INTERESSE

Verificar dados nulos ou faltantes.

Identificar e tratar outliers.

Padronizar formatos de data.

Codificar variáveis categóricas.

Corrigir inconsistências e erros de digitação.

Normalizar ou padronizar variáveis numéricas.

Remover colunas irrelevantes ou redundantes.

Eliminar dados duplicados.

Dividir colunas que contêm múltiplas informações.

Assegurar a consistência lógica dos dados.

Analisar a distribuição de valores únicos em colunas categóricas.

Agrupar categorias similares para simplificação.

Converter colunas de texto em categorias.

Agregar dados para criar métricas resumidas.

# TODO: ADICIONAR DADOS SINTÉTICOS

Desempenho Acadêmico e Áreas de Estudo: Correlacionar as notas e desempenho acadêmico com as áreas de estudo (como cursos ou especializações) para identificar alunos com alto desempenho em campos específicos que são demandados pelas empresas.

Habilidades Técnicas e Interesses Profissionais: Analisar a relação entre as habilidades técnicas (por exemplo, linguagens de programação, ferramentas de design, etc.) que os alunos possuem e seus interesses profissionais para encontrar correspondências com as necessidades das empresas.

Participação em Atividades Extracurriculares e Iniciativa: Investigar se a participação em clubes, sociedades estudantis e projetos extracurriculares está correlacionada com a proatividade e iniciativa, qualidades muitas vezes procuradas para posições de estágio.

Projetos de Curso e Experiência Prática: Correlacionar os tipos de projetos realizados durante o curso (como projetos de final de curso, estágios curriculares, pesquisas, etc.) com as áreas de interesse das empresas para recomendar alunos com experiência prática relevante.

Idiomas e Mercados Internacionais: Para empresas com operações internacionais, pode ser útil correlacionar a proficiência em idiomas com os mercados onde a empresa opera para recomendar alunos que possam se comunicar efetivamente em diferentes regiões.

Soft Skills e Cultura da Empresa: Analisar soft skills reportadas ou avaliadas (como trabalho em equipe, comunicação, liderança) e correlacioná-las com a cultura e valores das empresas parceiras para garantir uma boa integração cultural.

Disponibilidade e Locais de Estágio: Correlacionar a disponibilidade dos alunos (horários, período do estágio) e sua localização geográfica com as oportunidades de estágio disponíveis para recomendar alunos que se encaixam logisticamente nas necessidades das empresas.

Feedback de Estágios Anteriores: Se houver dados disponíveis, analisar o feedback de supervisores de estágios anteriores para identificar alunos que tiveram bom desempenho em ambientes profissionais.

In [26]:
# Encerra a sessão spark
spark.stop()